# Clasificación de Frutas Frescas vs. Podridas con ResNet34 - Análisis Profundo

Este notebook mejorado incluye un análisis exhaustivo del modelo, visualizaciones avanzadas y técnicas para mejorar la reproducibilidad y rendimiento.

In [ ]:
# Configuración inicial mejorada
import random
import numpy as np
import kagglehub
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
from PIL import Image
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Fijar semillas para reproducibilidad
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

print("✅ Configuración inicial completada con semilla fija")

## Análisis de Arquitectura ResNet34

La arquitectura ResNet34 utiliza conexiones residuales que permiten entrenar redes muy profundas (34 capas en este caso). Para nuestro problema de clasificación binaria:

- **Capas congeladas**: Todas excepto la última capa FC
- **Transfer Learning**: Aprovecha patrones aprendidos en ImageNet
- **Capa FC personalizada**: Adaptada a nuestras 2 clases (fresco/podrido)

In [ ]:
# Configuración mejorada del modelo
def initialize_model(num_classes):
    model = models.resnet34(pretrained=True)
    
    # Congelar parámetros inicialmente
    for param in model.parameters():
        param.requires_grad = False
    
    # Reemplazar capa FC
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, num_classes)
    
    # Descongelar últimas capas para fine-tuning
    for name, param in model.named_parameters():
        if 'layer4' in name or 'fc' in name:
            param.requires_grad = True
    
    return model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = initialize_model(2).to(device)

# Mostrar parámetros entrenables
print("\n🔍 Parámetros entrenables:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"- {name}")

## Data Augmentation Mejorada

Añadimos transformaciones adicionales para mejorar la generalización del modelo:

In [ ]:
# Transformaciones mejoradas
img_size = 224

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(img_size, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

print("✅ Transformaciones con data augmentation")

## Entrenamiento con Early Stopping y LR Scheduler

In [ ]:
# Configuración mejorada de entrenamiento
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.1)

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=15):
    best_val_loss = float('inf')
    patience = 3
    counter = 0
    
    train_losses = []
    val_losses = []
    val_accuracies = []
    
    for epoch in range(num_epochs):
        # Fase de entrenamiento
        model.train()
        running_loss = 0.0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Fase de validación
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                val_loss += criterion(outputs, labels).item()
                
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        # Calcular métricas
        train_loss = running_loss/len(train_loader)
        val_loss /= len(val_loader)
        accuracy = 100*correct/total
        
        # Actualizar scheduler
        scheduler.step(val_loss)
        
        # Guardar métricas
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        val_accuracies.append(accuracy)
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
        
        print(f"Epoch {epoch+1}/{num_epochs} - "
              f"Train Loss: {train_loss:.4f} - "
              f"Val Loss: {val_loss:.4f} - "
              f"Accuracy: {accuracy:.2f}% - "
              f"LR: {optimizer.param_groups[0]['lr']:.6f}")
    
    return train_losses, val_losses, val_accuracies

## Visualizaciones Mejoradas

In [ ]:
def plot_metrics(train_losses, val_losses, val_accuracies):
    plt.figure(figsize=(15, 5))
    
    # Gráfico de pérdidas
    plt.subplot(1, 3, 1)
    plt.plot(train_losses, label='Entrenamiento')
    plt.plot(val_losses, label='Validación')
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida')
    plt.title('Curva de Pérdida')
    plt.legend()
    
    # Gráfico de precisión
    plt.subplot(1, 3, 2)
    plt.plot(val_accuracies, color='green')
    plt.xlabel('Épocas')
    plt.ylabel('Precisión (%)')
    plt.title('Precisión en Validación')
    
    # Matriz de confusión
    plt.subplot(1, 3, 3)
    model.load_state_dict(torch.load('best_model.pth'))
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    cm = confusion_matrix(all_labels, all_preds)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, 
                yticklabels=class_names)
    plt.xlabel('Predicho')
    plt.ylabel('Verdadero')
    plt.title('Matriz de Confusión')
    
    plt.tight_layout()
    plt.show()

## Flujo Principal Mejorado

In [ ]:
try:
    # Descargar y cargar datos
    path = kagglehub.dataset_download("sriramr/fruits-fresh-and-rotten-for-classification")
    train_data_dir = os.path.join(path, 'dataset', 'train')
    val_data_dir = os.path.join(path, 'dataset', 'test')
    
    train_dataset = datasets.ImageFolder(root=train_data_dir, transform=train_transform)
    val_dataset = datasets.ImageFolder(root=val_data_dir, transform=val_transform)
    
    class_names = train_dataset.classes
    print(f"🔍 Clases detectadas: {class_names}")
    
    # DataLoaders
    batch_size = 64  # Reducido para permitir data augmentation
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
    # Entrenamiento
    train_losses, val_losses, val_accuracies = train_model(
        model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=20)
    
    # Visualización
    plot_metrics(train_losses, val_losses, val_accuracies)
    
except Exception as e:
    print(f"❌ Error: {e}")

## Interpretación de Resultados

1. **Curvas de Entrenamiento**:
   - Pérdida de entrenamiento debería disminuir consistentemente
   - Pérdida de validación debería seguir una tendencia similar
   - Grandes brechas indican sobreajuste

2. **Matriz de Confusión**:
   - Diagonal principal muestra clasificaciones correctas
   - Otras celdas muestran confusiones entre clases

3. **Learning Rate**:
   - Reducción automática cuando val_loss no mejora
   - Mejora la fine-tuning en etapas finales